In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# API Key
from config import weather_api_key

In [2]:
# Build the basic URL
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

620

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                        "Lat": city_lat,
                        "Lng": city_lng,
                        "Max Temp": city_max_temp,
                        "Humidity": city_humidity,
                        "Cloudiness": city_clouds,
                        "Wind Speed": city_wind,
                        "Country": city_country,
                        "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | fayaoue
Processing Record 2 of Set 1 | ilulissat
Processing Record 3 of Set 1 | altonia
City not found. Skipping...
Processing Record 4 of Set 1 | havre-saint-pierre
Processing Record 5 of Set 1 | georgetown
Processing Record 6 of Set 1 | garowe
Processing Record 7 of Set 1 | leningradskiy
Processing Record 8 of Set 1 | mar del plata
Processing Record 9 of Set 1 | jamestown
Processing Record 10 of Set 1 | wenling
Processing Record 11 of Set 1 | chokurdakh
Processing Record 12 of Set 1 | vao
Processing Record 13 of Set 1 | hilo
Processing Record 14 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 15 of Set 1 | busselton
Processing Record 16 of Set 1 | cayenne
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | kapaa
Processing Record 19 of Set 1 | vilyuysk
Processing Record 20 of Set 1 | erenhot
Processing Record 21 of Set 1 | mataura
Processing Record 

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Fayaoue,-20.6500,166.5333,82.80,84,96,27.22,NC,2022-03-01 02:11:40
1,Ilulissat,69.2167,-51.1000,7.79,57,65,11.30,GL,2022-03-01 02:11:40
2,Havre-Saint-Pierre,50.2334,-63.5986,-6.14,90,6,8.97,CA,2022-03-01 02:09:41
3,Georgetown,5.4112,100.3354,78.75,80,95,4.68,MY,2022-03-01 02:08:05
4,Garowe,8.4054,48.4845,65.70,87,94,10.60,SO,2022-03-01 02:11:41
5,Leningradskiy,69.3833,178.4167,17.38,74,49,9.82,RU,2022-03-01 02:11:41
6,Mar Del Plata,-38.0023,-57.5575,55.85,52,5,10.76,AR,2022-03-01 02:08:50
7,Jamestown,42.0970,-79.2353,25.61,83,100,3.80,US,2022-03-01 02:09:03
8,Wenling,28.3669,121.3600,61.05,82,61,8.88,CN,2022-03-01 02:11:41
9,Chokurdakh,70.6333,147.9167,-7.91,98,97,4.56,RU,2022-03-01 02:11:42


In [13]:
new_column_order = ["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]
city_data_df=city_data_df[new_column_order]  
city_data_df 

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Fayaoue,NC,2022-03-01 02:11:40,-20.6500,166.5333,82.80,84,96,27.22
1,Ilulissat,GL,2022-03-01 02:11:40,69.2167,-51.1000,7.79,57,65,11.30
2,Havre-Saint-Pierre,CA,2022-03-01 02:09:41,50.2334,-63.5986,-6.14,90,6,8.97
3,Georgetown,MY,2022-03-01 02:08:05,5.4112,100.3354,78.75,80,95,4.68
4,Garowe,SO,2022-03-01 02:11:41,8.4054,48.4845,65.70,87,94,10.60
...,...,...,...,...,...,...,...,...,...
563,Damghan,IR,2022-03-01 02:26:36,36.1683,54.3480,43.59,61,81,1.66
564,Marquard,ZA,2022-03-01 02:26:36,-28.6645,27.4305,58.91,86,89,3.76
565,Ternate,ID,2022-03-01 02:26:36,0.8000,127.4000,83.84,69,78,2.86
566,Mayumba,GA,2022-03-01 02:26:37,-3.4320,10.6554,78.26,89,100,4.54


In [14]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")